In [6]:
import tensorflow as tf
from tensorflow.keras.callbacks import CSVLogger, ModelCheckpoint
from tensorflow.keras.utils import to_categorical
import os
from scipy.io import wavfile
import pandas as pd
import numpy as np
from sklearn.utils.class_weight import compute_class_weight
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from Models import Conv1D, Conv2D, LSTM
from tqdm import tqdm
from glob import glob
import argparse
import warnings


class DataGenerator(tf.keras.utils.Sequence):
    def __init__(self, wav_paths, labels, sr, dt, n_classes,
                 batch_size=32, shuffle=True):
        self.wav_paths = wav_paths
        self.labels = labels
        self.sr = sr
        self.dt = dt
        self.n_classes = n_classes
        self.batch_size = batch_size
        self.shuffle = True
        self.on_epoch_end()


    def __len__(self):
        return int(np.floor(len(self.wav_paths) / self.batch_size))


    def __getitem__(self, index):
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        wav_paths = [self.wav_paths[k] for k in indexes]
        labels = [self.labels[k] for k in indexes]

        '''
        generate a batch of time data
        X: input data - normalized and splited sample files 
        Y: output classes
        '''
        X = np.empty((self.batch_size, int(self.sr*self.dt), 1), dtype=np.float32)
        Y = np.empty((self.batch_size, self.n_classes), dtype=np.float32)

        for i, (path, label) in enumerate(zip(wav_paths, labels)):
            rate, wav = wavfile.read(path)
            X[i,] = wav.reshape(-1, 1)
            Y[i,] = to_categorical(label, num_classes=self.n_classes)

        return X, Y


    def on_epoch_end(self):
        self.indexes = np.arange(len(self.wav_paths))
        if self.shuffle:
            np.random.shuffle(self.indexes)


def train(args):
    src_root = args.src_root
    sr = args.sample_rate
    dt = args.delta_time
    batch_size = args.batch_size
    model_type = args.model_type
    params = {'N_CLASSES':len(os.listdir(args.src_root)),
              'SR':sr,
              'DT':dt}
    models = {'conv1d':Conv1D(**params),
              'conv2d':Conv2D(**params),
              'lstm':  LSTM(**params)}
    assert model_type in models.keys(), '{} not an available model'.format(model_type)
    csv_path = os.path.join('./logs', '{}_history.csv'.format(model_type))

    wav_paths = glob('{}/**'.format(src_root), recursive=True)
    wav_paths = [x.replace(os.sep, '/') for x in wav_paths if '.wav' in x]
    classes = sorted(os.listdir(args.src_root))
    le = LabelEncoder()
    le.fit(classes)
    labels = [os.path.split(x)[0].split('/')[-1] for x in wav_paths]
    labels = le.transform(labels)
    
    # Train Test Split
    wav_train, wav_val, label_train, label_val = train_test_split(wav_paths,
                                                                  labels,
                                                                  test_size=0.1,
                                                                  random_state=0)

    assert len(label_train) >= args.batch_size, 'Number of train samples must be >= batch_size'
    if len(set(label_train)) != params['N_CLASSES']:
        warnings.warn('Found {}/{} classes in training data. Increase data size or change random_state.'.format(len(set(label_train)), params['N_CLASSES']))
    if len(set(label_val)) != params['N_CLASSES']:
        warnings.warn('Found {}/{} classes in validation data. Increase data size or change random_state.'.format(len(set(label_val)), params['N_CLASSES']))

    # Train Data
    tg = DataGenerator(wav_train, label_train, sr, dt,
                       params['N_CLASSES'], batch_size=batch_size)
    
    # Validation Data
    vg = DataGenerator(wav_val, label_val, sr, dt,
                       params['N_CLASSES'], batch_size=batch_size)
    
    model = models[model_type]
    cp = ModelCheckpoint('./models/{}.h5'.format(model_type), monitor='val_loss',
                         save_best_only=True, save_weights_only=False,
                         mode='auto', save_freq='epoch', verbose=1)
    csv_logger = CSVLogger(csv_path, append=False)
    
    model.summary()
    
    model.fit(tg, validation_data=vg,
              epochs=10, verbose=1,
              callbacks=[csv_logger, cp])

In [7]:
if __name__ == '__main__':

    parser = argparse.ArgumentParser(description='Audio Classification Training')
    parser.add_argument('--model_type', type=str, default='lstm',
                        help='model to run. i.e. conv1d, conv2d, lstm')
    parser.add_argument('--src_root', type=str, default='../Thingy52/clean',
                        help='directory of audio files in total duration')
    parser.add_argument('--batch_size', type=int, default=16,
                        help='batch size')
    parser.add_argument('--delta_time', '-dt', type=float, default=1.0,
                        help='time in seconds to sample audio')
    parser.add_argument('--sample_rate', '-sr', type=int, default=16000,
                        help='sample rate of clean audio')
    args, _ = parser.parse_known_args()

    train(args)

C:\Users\Ahhyun Yuh\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:94: UserWarning: Found 8/10 classes in training data. Increase data size or change random_state.
C:\Users\Ahhyun Yuh\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:96: UserWarning: Found 8/10 classes in validation data. Increase data size or change random_state.


Model: "long_short_term_memory"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
stft_8_input (InputLayer)       [(None, 16000, 1)]   0                                            
__________________________________________________________________________________________________
stft_8 (STFT)                   (None, 100, 257, 1)  0           stft_8_input[0][0]               
__________________________________________________________________________________________________
magnitude_8 (Magnitude)         (None, 100, 257, 1)  0           stft_8[0][0]                     
__________________________________________________________________________________________________
apply_filterbank_8 (ApplyFilter (None, 100, 128, 1)  0           magnitude_8[0][0]                
_____________________________________________________________________________

In [8]:
if __name__ == '__main__':

    parser = argparse.ArgumentParser(description='Audio Classification Training')
    parser.add_argument('--model_type', type=str, default='conv1d',
                        help='model to run. i.e. conv1d, conv2d, lstm')
    parser.add_argument('--src_root', type=str, default='../Thingy52/clean',
                        help='directory of audio files in total duration')
    parser.add_argument('--batch_size', type=int, default=16,
                        help='batch size')
    parser.add_argument('--delta_time', '-dt', type=float, default=1.0,
                        help='time in seconds to sample audio')
    parser.add_argument('--sample_rate', '-sr', type=int, default=16000,
                        help='sample rate of clean audio')
    args, _ = parser.parse_known_args()

    train(args)

C:\Users\Ahhyun Yuh\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:94: UserWarning: Found 8/10 classes in training data. Increase data size or change random_state.
C:\Users\Ahhyun Yuh\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:96: UserWarning: Found 8/10 classes in validation data. Increase data size or change random_state.


Model: "1d_convolution"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
stft_9_input (InputLayer)    [(None, 16000, 1)]        0         
_________________________________________________________________
stft_9 (STFT)                (None, 100, 257, 1)       0         
_________________________________________________________________
magnitude_9 (Magnitude)      (None, 100, 257, 1)       0         
_________________________________________________________________
apply_filterbank_9 (ApplyFil (None, 100, 128, 1)       0         
_________________________________________________________________
magnitude_to_decibel_9 (Magn (None, 100, 128, 1)       0         
_________________________________________________________________
batch_norm (LayerNormalizati (None, 100, 128, 1)       256       
_________________________________________________________________
td_conv_1d_tanh (TimeDistrib (None, 100, 125, 8)    

In [9]:
if __name__ == '__main__':

    parser = argparse.ArgumentParser(description='Audio Classification Training')
    parser.add_argument('--model_type', type=str, default='conv2d',
                        help='model to run. i.e. conv1d, conv2d, lstm')
    parser.add_argument('--src_root', type=str, default='../Thingy52/clean',
                        help='directory of audio files in total duration')
    parser.add_argument('--batch_size', type=int, default=16,
                        help='batch size')
    parser.add_argument('--delta_time', '-dt', type=float, default=1.0,
                        help='time in seconds to sample audio')
    parser.add_argument('--sample_rate', '-sr', type=int, default=16000,
                        help='sample rate of clean audio')
    args, _ = parser.parse_known_args()

    train(args)

C:\Users\Ahhyun Yuh\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:94: UserWarning: Found 8/10 classes in training data. Increase data size or change random_state.
C:\Users\Ahhyun Yuh\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:96: UserWarning: Found 8/10 classes in validation data. Increase data size or change random_state.


Model: "2d_convolution"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
stft_13_input (InputLayer)   [(None, 16000, 1)]        0         
_________________________________________________________________
stft_13 (STFT)               (None, 100, 257, 1)       0         
_________________________________________________________________
magnitude_13 (Magnitude)     (None, 100, 257, 1)       0         
_________________________________________________________________
apply_filterbank_13 (ApplyFi (None, 100, 128, 1)       0         
_________________________________________________________________
magnitude_to_decibel_13 (Mag (None, 100, 128, 1)       0         
_________________________________________________________________
batch_norm (LayerNormalizati (None, 100, 128, 1)       256       
_________________________________________________________________
conv2d_tanh (Conv2D)         (None, 100, 128, 8)    